
Install the libraries

```
!pip install --upgrade torch transformers accelerate
```

In [ ]:
!pip install --upgrade torch transformers accelerate

In [ ]:
import os
import shutil
from pathlib import Path

import torch
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer
from utils import recreate_folder

In [ ]:
model_name = "gpt2"
project_name = os.environ['DOMINO_PROJECT_NAME']
save_path = f"/home/ubuntu/{model_name}"  # Set the directory to save the model
recreate_folder(save_path)


In [ ]:
# Load the model and tokenizer from Hugging Face
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Save them locally
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved locally at {save_path}")

## Test the downloaded model locally

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

def load_local_llm(model_path):
    """Load a locally stored LLM model from disk."""
    #tokenizer = AutoTokenizer.from_pretrained(model_path)
    #model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.float16, 
                                             device_map="cpu")

    print(f"{model_name} loaded in float16 on CPU")
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_text(llm_pipeline, prompt, max_length=50):
    """Generate text using the loaded LLM."""
    response = llm_pipeline(prompt, max_length=max_length, do_sample=True, temperature=0.7)
    return response[0]['generated_text']

In [ ]:
model_path = save_path  # Change this to your local model directory
model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.float16, 
                                             device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a text-generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text
prompt = "Once upon a time in a distant land,"
output = text_generator(prompt, max_length=50, do_sample=True)
print(output[0]['generated_text'])